In [69]:
import spacy
from spacy.matcher import Matcher
from spacy import displacy
from spacy.matcher import DependencyMatcher

nlp = spacy.load('en_core_web_md')


def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
            print(ent.text+' - '+ent.label_+' - ' +
                  str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')

# Ruler
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {'label': "ORG", "pattern": "Saba Portal"},
    {'label': "ORG", "pattern": "Saba portal"},
    {'label': "ORG", "pattern": "saba portal"},
    {'label': "ORG", "pattern": "Saba"},
    {'label': "ORG", "pattern": "saba"},
]
ruler.add_patterns(patterns)

# Matcher
matcher = Matcher(nlp.vocab)
saba_patterns = [[{"LOWER": "saba"}]]
matcher.add("Saba", saba_patterns)


In [70]:
# Dependency Matcher
dep_matcher = DependencyMatcher(nlp.vocab)
dep_pattern = [
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": ">",
            "RIGHT_ID": "negation",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
        {
            "LEFT_ID": "saba_action",
            "REL_OP": "<<",
            "RIGHT_ID": "aux_pre_neg",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_pre_neg",
            "REL_OP": ">",
            "RIGHT_ID": "aux_neg",
            "RIGHT_ATTRS": {"DEP": "neg"}
        },
    ],
    [
        {
            "RIGHT_ID": "anchor_saba",
            "RIGHT_ATTRS": {"LOWER": "saba"}
        },
        {
            "LEFT_ID": "anchor_saba",
            "REL_OP": "<<",
            "RIGHT_ID": "aux_unable",
            "RIGHT_ATTRS": {"POS": "AUX"}
        },
        {
            "LEFT_ID": "aux_unable",
            "REL_OP": ">",
            "RIGHT_ID": "unable",
            "RIGHT_ATTRS": {"LOWER": "unable"}
        },
        {
            "LEFT_ID": "unable",
            "REL_OP": ">>",
            "RIGHT_ID": "saba_action",
            "RIGHT_ATTRS": {"POS": "VERB"}
        },
    ],
]
dep_matcher.add("saba_dep", patterns=dep_pattern)


In [90]:
global_scenario = 1
global_domain_matches = None
global_dep_matches = None


def create_ticket():
    choice = input(
        "Do you want to proceed to create a manual ticket? (Yes/No)")
    if (choice.lower() == "yes"):
        print(
            f"Creating a ticket now for your query:\n\nPlease wait for a moment.")
    else:
        return


def scenarios(scenario):
    global global_scenario
    if (scenario == 0):
        choice = input(
            "We are unable to understand the problem. Do you want to try describing your problem again? (yes/no)")
        if (choice.lower() == "yes"):
            scenario = global_scenario
        else:
            create_ticket()
            return

    if (scenario == 1):
        prompt = input(
            "Please specify the domain of the problem being faced. Eg. Saba portal, Learning portal")
        selfhealer(prompt, 0)
        return

    if (scenario == 2):
        prompt = input(
            "Please describe the problem in detail or provide us with the error description and error code.")
        selfhealer(prompt, 0)
        return

    if (scenario == 3):

        return


def lms_access(action):
    print(
        "Accessing LMS databases to assess a fix {action} issue in Saba portal")


def selfhealer(prompt, scenario=global_scenario):
    global global_scenario, global_domain_matches, global_dep_matches
    doc = nlp(prompt)

    # Scenario 1 check
    if (global_scenario == 1):
        domain_matches = matcher(doc)
        if (len(domain_matches) == 0):
            scenarios(scenario)
            return
        else:
            global_domain_matches = domain_matches
            global_scenario = 2
            scenario = 2
            print(global_domain_matches)

    if (global_scenario == 2):
        dep_matches = dep_matcher(doc)
        if (len(dep_matches) == 0):
            scenarios(scenario)
            return
        else:
            global_dep_matches = dep_matches
            global_scenario = 3
            scenario = 3

    actions = []
    for match in global_dep_matches:
        temp_matches = match[1]
        for sub_match in temp_matches:
            if ((sub_match not in actions) and (str(doc[sub_match].pos_) == "VERB")):
                actions.append(doc[sub_match].lemma_)

    for action in actions:
        if (str(action).lower() in ["login", "log", "access", "enter"]):
            lms_access(str(action).lower())


selfhealer("I am facing issues accessing saba portal")
print(global_scenario)


[(2914936248315507649, 5, 6)]


2


In [94]:
doc = nlp("I am facing issues accessing saba portal")
dep_matches = dep_matcher(doc)

actions = []
for match in dep_matches:
    temp_matches = match[1]
    for sub_match in temp_matches:
        if ((sub_match not in actions) and (str(doc[sub_match].pos_) == "VERB")):
            actions.append(doc[sub_match].lemma_)

for action in actions:
    if (str(action).lower() in ["login", "log", "access", "enter"]):
        lms_access(str(action).lower())

In [ ]:
for token in doc:
    print(f"{token.text:{10}} {token.pos_:{10}} ({token.dep_:{10}} {spacy.explain(token.dep_):{25}})  {token.head}")
    

In [ ]:
doc = nlp("I am facing issues accessing saba portal")
# show_ents(doc)
for sent in doc.sents:
    print (displacy.render(sent, style = "dep"))
    